In [1]:
from jesse import jesse
jesse.SetOptions()
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame
from IPython.display import clear_output

import pybaseball as _pybaseball
import preprocess
from preprocess import service, model, dates
#from process import batter_processing


/Users/downing.jesse/Projects/MLB_statTRAX/python
Batter data is current.


In [5]:
# this was build by preprocess
batter_data = pd.read_csv('../data/csv/raw_batter_stats.csv')

In [2]:
#batter_data[batter_data.Name == 'J.D. Martinez'].head(5)

### Get raw stats for a batter

In [ ]:
#gallo_stats = mlb.statcast_batter('2019-04-01', '2019-05-01', id)
gallo_stats = mlb.statcast_batter(start_dt='2019-02-20', end_dt=today, player_id=id)

### Grab all batted ball events for a single day for a single player

In [ ]:
gallo_stats[(gallo_stats.game_date == '2019-05-01') & (~gallo_stats.events.isnull())]

In [ ]:
gallo_stats.events.value_counts()

## What other player sources exist?
* Lahman
** batting stats by year
** updated at the end of each season (current is through 2018)
* fangraps
* baseball reference
* baseball savants